<a href="https://colab.research.google.com/github/ruforavishnu/attempting_kaggle_2025_santa_competition/blob/main/03_spatial_partitioning_and_efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
from shapely.geometry import Polygon



In [ ]:
def get_bbox(poly):
  min_x , min_y, max_x, max_y = poly.bounds

  return min_x, min_y, max_x, max_y


In [ ]:
def world_to_cell(x,y, cell_size):
  cx = int(math.floor(x/cell_size))
  cy = int(math.floor(y/cell_size))


  return cx, cy



In [ ]:
def bbox_to_cells(bbox, cell_size):
  min_x, min_y, max_x, max_y = bbox

  cx_min, cy_min = world_to_cell(min_x, min_y, cell_size)
  cx_max, cy_max = world_to_cell(max_x, max_y, cell_size)

  cells = []

  for cx in range(cx_min, cx_max+1):
    for cy in range(cy_min, cy_max+1):
      cells.append(  (cx,cy)  )

  return cells


In [3]:
from collections import defaultdict

In [18]:
class SpatialHashGrid:

  def __init__(self, cell_size):
    self.cell_size = cell_size
    self.grid = defaultdict(list)


  def _cell_coords(self, x, y):
    cell_x = math.floor( x / self.cell_size)
    cell_y = math.floor( y / self.cell_size)

    return (cell_x, cell_y)



  def add_object(self, obj, x, y):
    cell = self._cell_coords(x,y)
    self.grid[cell].append( (obj, x, y)  )


  def remove_object(self, obj, x, y):
    cell = self._cell_coords(x,y)

    if cell in self.grid:
      self.grid[cell] = [o for o in self.grid[cell] if o[0] != obj ]

      if not self.grid[cell]:
        del self.grid[cell]



  def move_object(self, obj, old_x, old_y, new_x, new_y):

    old_cell = self._cell_coords(old_x, old_y)
    new_cell = self._cell_coords(new_x, new_y)


    if old_cell != new_cell:
      self.remove_object(obj, old_x, old_y)
      self.add_object(obj, new_x, new_y)


  def query_radius(self, x, y, radius):

    min_x = x - radius
    max_x = x + radius
    min_y = y - radius
    max_y = y + radius


    cell_min = self._cell_coords(min_x, min_y)
    cell_max = self._cell_coords(max_x, max_y)


    results = []


    for cx in range(cell_min[0],    cell_max[0] + 1):
      for cy in range(cell_min[1],  cell_max[1] + 1):
        cell = (cx, cy)


        if cell in self.grid:
          for obj, ox, oy in self.grid[cell]:

            if (ox - x)**2 + (oy - y)**2 <= radius**2:
              results.append(obj)


    return results



### Insert 1000 Random Objects + Test Query Speed

In [19]:
import random
import time
from collections import defaultdict

cell_size = 50
grid = SpatialHashGrid(cell_size)

In [20]:
import math

In [21]:
NUM_TREES = 1000
trees = []


for i in range(NUM_TREES):
  obj_name = f"tree_{i}"


  x = random.uniform(0, 2000)
  y = random.uniform(0, 2000)



  trees.append( (obj_name, x, y)  )

  grid.add_object(obj_name, x, y)



In [22]:
query_x = random.uniform(0, 2000)
query_y = random.uniform(0, 2000)

query_radius = 50




print(f"Query point: ({query_x:.1f} , {query_y:.1f}) radius = {query_radius}")


Query point: (838.3 , 1257.7) radius = 50


##### NAIVE search timing

In [23]:
start = time.time()

naive_results = []
for obj_name, x, y in trees:
  if (x - query_x)**2 + (y - query_y)**2 <= query_radius**2:
    naive_results.append(obj_name)


end = time.time()
naive_time = end - start



In [24]:
print(f'naive time: {naive_time}')

naive time: 0.0011851787567138672


##### GRID search timing

In [25]:
start = time.time()
grid_results = grid.query_radius(query_x, query_y, query_radius)
end = time.time()
grid_time = end - start

In [26]:
print(f'Grid time: {grid_time}')

Grid time: 0.00012612342834472656


### Results

In [27]:
print(f'\nRESULTS')
print(f'Naive method found: {len(naive_results)} trees')
print(f'Grid method found: {len(grid_results)} trees')
print()

print(f'Naive method time: {naive_time*1000:.4f} ms')
print(f'Grid method time: {grid_time*1000:.4f} ms')


RESULTS
Naive method found: 3 trees
Grid method found: 3 trees

Naive method time: 1.1852 ms
Grid method time: 0.1261 ms


## See the milliseconds saved?! Close to 1 milli-second profit achieved.

#Visualize the Grid Cells that Get Queried

In [ ]:
import matplotlib.pyplot as plt
import math



def plot_grid(grid, trees, query_x, query_y, radius, world_size=2000):
  cell = grid.cell_size
